In [ ]:
from datetime import datetime
start_time = datetime.now()

In [ ]:
import pandas as pd
import numpy as np
import torch
import os
import random
import torchvision
from torchvision import transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from sklearn.model_selection import train_test_split
from PIL import Image , ImageDraw

In [ ]:
train = pd.read_csv("D:\\projek pka\\global-wheat-detection\\train.csv")
train.head(10)

image_id  width  height                          bbox   source
0  b6ab77fd7   1024    1024    [834.0, 222.0, 56.0, 36.0]  usask_1
1  b6ab77fd7   1024    1024   [226.0, 548.0, 130.0, 58.0]  usask_1
2  b6ab77fd7   1024    1024   [377.0, 504.0, 74.0, 160.0]  usask_1
3  b6ab77fd7   1024    1024   [834.0, 95.0, 109.0, 107.0]  usask_1
4  b6ab77fd7   1024    1024   [26.0, 144.0, 124.0, 117.0]  usask_1
5  b6ab77fd7   1024    1024  [569.0, 382.0, 119.0, 111.0]  usask_1
6  b6ab77fd7   1024    1024     [52.0, 602.0, 82.0, 45.0]  usask_1
7  b6ab77fd7   1024    1024   [627.0, 302.0, 122.0, 75.0]  usask_1
8  b6ab77fd7   1024    1024    [412.0, 367.0, 68.0, 82.0]  usask_1
9  b6ab77fd7   1024    1024   [953.0, 220.0, 56.0, 103.0]  usask_1

In [ ]:
coord = pd.DataFrame(list(train.bbox.apply(lambda x : x[1:-1].split(",")).values), columns=["x1", "y1", "w", "h"])

In [ ]:
df = pd.concat([train, coord], axis=1)

In [ ]:
df['x1'] = pd.to_numeric(df['x1'])
df['y1'] = pd.to_numeric(df['y1'])
df['w'] = pd.to_numeric(df['w'])
df['h'] = pd.to_numeric(df['h'])

In [ ]:
df['x2']=df['x1']+df['w']
df['y2']=df['y1']+df['h']

In [ ]:
df

image_id  width  height                         bbox     source  \
0       b6ab77fd7   1024    1024   [834.0, 222.0, 56.0, 36.0]    usask_1   
1       b6ab77fd7   1024    1024  [226.0, 548.0, 130.0, 58.0]    usask_1   
2       b6ab77fd7   1024    1024  [377.0, 504.0, 74.0, 160.0]    usask_1   
3       b6ab77fd7   1024    1024  [834.0, 95.0, 109.0, 107.0]    usask_1   
4       b6ab77fd7   1024    1024  [26.0, 144.0, 124.0, 117.0]    usask_1   
...           ...    ...     ...                          ...        ...   
147788  5e0747034   1024    1024    [64.0, 619.0, 84.0, 95.0]  arvalis_2   
147789  5e0747034   1024    1024  [292.0, 549.0, 107.0, 82.0]  arvalis_2   
147790  5e0747034   1024    1024  [134.0, 228.0, 141.0, 71.0]  arvalis_2   
147791  5e0747034   1024    1024   [430.0, 13.0, 184.0, 79.0]  arvalis_2   
147792  5e0747034   1024    1024   [875.0, 740.0, 94.0, 61.0]  arvalis_2   

           x1     y1      w      h     x2     y2  
0       834.0  222.0   56.0   36.0  890.0  258.0  
1       226.0  548.0  130.0   58.0  356.0  606.0  
2       377.0  504.0   74.0  160.0  451.0  664.0  
3       834.0   95.0  109.0  107.0  943.0  202.0  
4        26.0  144.0  124.0  117.0  150.0  261.0  
...       ...    ...    ...    ...    ...    ...  
147788   64.0  619.0   84.0   95.0  148.0  714.0  
147789  292.0  549.0  107.0   82.0  399.0  631.0  
147790  134.0  228.0  141.0   71.0  275.0  299.0  
147791  430.0   13.0  184.0   79.0  614.0   92.0  
147792  875.0  740.0   94.0   61.0  969.0  801.0  

[147793 rows x 11 columns]

In [ ]:
df.drop(["width", "height", "bbox", "source", "w", "h"], axis=1, inplace=True)

In [ ]:
df.head()

image_id     x1     y1     x2     y2
0  b6ab77fd7  834.0  222.0  890.0  258.0
1  b6ab77fd7  226.0  548.0  356.0  606.0
2  b6ab77fd7  377.0  504.0  451.0  664.0
3  b6ab77fd7  834.0   95.0  943.0  202.0
4  b6ab77fd7   26.0  144.0  150.0  261.0

In [ ]:
unique_imgs = df.image_id.unique()
unique_imgs

array(['b6ab77fd7', 'b53afdf5c', '7b72ea0fb', ..., 'a5c8d5f5c',
       'e6b5e296d', '5e0747034'], dtype=object)

In [ ]:
class wheatData(torch.utils.data.Dataset):
    def __init__(self, df, unique_imgs, indicies):
        self.df = df
        self.unique_imgs = unique_imgs
        self.indicies = indicies

    def __len__(self):
        return len(self.indicies)

    def __getitem__(self, idx):
        image_name = self.unique_imgs[self.indicies[idx]]
        boxes = self.df[self.df.image_id == image_name].values[:, 1:].astype("float")
        img = Image.open("D:\\global-wheat-detection\\train\\" + image_name + ".jpg").convert('RGB')
        labels = torch.ones(boxes.shape[0], dtype=torch.int64)
        target = {}
        target ["boxes"] = torch.tensor(boxes)
        target ["label"] = labels
        return T.ToTensor() (img), target

In [ ]:
train_inds , val_inds = train_test_split(range(unique_imgs.shape[0]),test_size=0.1)

In [ ]:
def custom_collate(data):
    return data

In [ ]:
train_dl = torch.utils.data.DataLoader(wheatData(df,unique_imgs,train_inds), batch_size=8, shuffle=True, collate_fn=custom_collate, num_workers=4, pin_memory=True if torch.cuda.is_available() else False)
val_dl = torch.utils.data.DataLoader(wheatData(df,unique_imgs,val_inds), batch_size=8, shuffle=True, collate_fn=custom_collate, num_workers=4, pin_memory=True if torch.cuda.is_available() else False)

In [ ]:
accumulation_steps = 4

In [ ]:
mymodel = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights = 'DEFAULT', pretrained=True)
num_classes = 2 # 1 untuk mendeteksi gandum dan 0 untuk background
in_features = mymodel.roi_heads.box_predictor.cls_score.in_features
mymodel.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
optimizer = torch.optim.SGD(mymodel.parameters(), lr=0.001, momentum=0.9)
num_epochs = 5

In [ ]:
print(torch.cuda.is_available())
print(torch.version.cuda)


True
12.1


#Model

In [ ]:
mymodel.to(device)
for epochs in range(num_epochs):
    epoch_loss = 0
    print("epoch: " , epochs)
    for data in train_dl:
        imgs = []
        targets = []
        # print("Data ke:" ,data)
        for d in data:
            imgs.append(d[0].to(device))
            targ = {}
            targ['boxes'] = d[1]['boxes'].to(device)
            targ['labels'] = d[1]['label'].to(device)
            targets.append(targ)
            # print("d ke: ", d)
        loss_dict = mymodel(imgs,targets)
        loss = sum(v for v in loss_dict.values())
        epoch_loss += loss.cpu().detach().numpy()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epochs + 1}, Loss: {epoch_loss}")

epoch:  0


In [ ]:
mymodel.eval()
data = iter(val_dl).__next__()

In [ ]:
img = data[0][0]
boxes = data[0][1]['boxes']
labels = data[0][1]['label']

In [ ]:
output = mymodel([img.to(device)])

KeyboardInterrupt: 

In [ ]:
output

NameError: name 'output' is not defined

In [ ]:
out_bbox = output[0]['boxes']
out_score = output[0]['scores']

NameError: name 'output' is not defined

In [ ]:
keep = torchvision.ops.nms(out_bbox,out_score,0.45)

NameError: name 'out_bbox' is not defined

In [ ]:
out_bbox.shape, keep.shape

In [ ]:
im = (img.permute(1,2,0).cpu().detach().numpy()*255).astype('uint8')
im

In [ ]:
vsample = Image.fromarray(im)
draw = ImageDraw.Draw(vsample)
for box in boxes :
    draw.rectangle(list(box),fill=None,outline='red')
vsample

In [ ]:
# Save the model state
torch.save(model.state_dict(), 'modified_fasterrcnn_resnet50_fpn.pth')

In [ ]:
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:01.877979
